# Train model LARS Lasso

## Prepare data

In [1]:
import pandas as pd
import pickle
from sklearn.model_selection import TimeSeriesSplit

from sklearn.linear_model import LassoLars
from sklearn.metrics import r2_score

INPUT_DATA_FOLDER = "data/source"
ALL_MERGED_DATA = "all_merged_data.csv"

MODEL_FOLDER = "model"
MODEL_FILE_NAME = "lars_lasso_model.pkl"

FEATURES_TO_INCLUDE = [
    'city_resolvedAddress',
    'day_temp',
    'day_humidity',
    'hour_windspeed',
    'hour_conditions',
    'event_start_hour',
    'event_num_regions',
    'event_num_alarms_24h',
    'vectors'
]
TARGET_FEATURE = 'is_alarm'

# Load dataset from a CSV file
df = pd.read_csv(f"{INPUT_DATA_FOLDER}/{ALL_MERGED_DATA}", sep=";")

# Separate the target variable from the input features
X = df[FEATURES_TO_INCLUDE]
y = df[TARGET_FEATURE]

# Convert all columns to float
X = X.apply(pd.to_numeric, errors='coerce')
y = y.apply(pd.to_numeric, errors='coerce')

# Replace NaN values with default
X.fillna(0, inplace=True)
y.fillna(0, inplace=True)

# Split the dataset into training and testing sets
tss = TimeSeriesSplit(2)
train_index, test_index = list(tss.split(X))[-1]
X_train, X_test = X.iloc[train_index, :], X.iloc[test_index,:]
y_train, y_test = y.iloc[train_index], y.iloc[test_index]

# X_train[X_train['vectors'] > 0.0].head(3)
X_train.head(3)


,city_resolvedAddress,day_temp,day_humidity,hour_windspeed,hour_conditions,event_start_hour,event_num_regions,event_num_alarms_24h,vectors
0,0.0,2.6,83.7,15.5,0.0,0.0,0.0,0.0,0.0
1,0.0,2.6,83.7,14.8,0.0,0.0,0.0,0.0,0.0
2,0.0,2.6,83.7,14.4,0.0,0.0,0.0,0.0,0.0


## Train model

In [2]:
# Train a LARS Lasso model
model = LassoLars(alpha=0.1, max_iter=100, normalize=True)
model.fit(X_train, y_train)

# Make predictions on the testing set
y_pred = model.predict(X_test)

# Evaluate the model's performance using R-squared score
r2 = r2_score(y_test, y_pred)
print('R-squared score:', r2)


R-squared score: -0.03875234202857736


## Save model

In [3]:
with open(f"{MODEL_FOLDER}/{MODEL_FILE_NAME}", 'wb') as f:
    pickle.dump(model, f)

## Calculate confusion matrix

In [ ]:
from sklearn.metrics import confusion_matrix

y_pred = model.predict(X_test)
cm = confusion_matrix(y_test, y_pred)
print('Confusion Matrix:')
print(cm)